In [29]:
%%time
dict0 = {'a':[1, 2], 'b':['+', '*'], 'c':['!', '@']}
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print(f'a {a}, b {b}, c {c}')

a 1, b +, c !
a 1, b +, c @
a 1, b *, c !
a 1, b *, c @
a 2, b +, c !
a 2, b +, c @
a 2, b *, c !
a 2, b *, c @
CPU times: user 94 μs, sys: 5 μs, total: 99 μs
Wall time: 102 μs


In [31]:
%%time
from itertools import product
dict0 = {'a':[1, 2], 'b':['+', '*'], 'c':['!', '@']}
jobs = (dict(zip(dict0, i)) for i in product(*dict0.values()))
for i in jobs:
    print(i)

{'a': 1, 'b': '+', 'c': '!'}
{'a': 1, 'b': '+', 'c': '@'}
{'a': 1, 'b': '*', 'c': '!'}
{'a': 1, 'b': '*', 'c': '@'}
{'a': 2, 'b': '+', 'c': '!'}
{'a': 2, 'b': '+', 'c': '@'}
{'a': 2, 'b': '*', 'c': '!'}
{'a': 2, 'b': '*', 'c': '@'}
CPU times: user 336 μs, sys: 0 ns, total: 336 μs
Wall time: 310 μs


### Single thread vs Multithreading vs Multiprocessing

In [32]:
import numpy as np
def main0():
    r = np.random.normal(0, .01, size=(1000, 10000))
    t = barrierTouch(r)
    return

def barrierTouch(r, width=0.5):
    # find the index of the earliest barrier touch
    t, p = {}, np.log((1+r).cumprod(axis=0))
    for j in range(r.shape[1]):
        for i in range(r.shape[0]):
            if p[i,j] >= width or p[i, j] <= -width:
                t[j] = i
                continue
    return t

if __name__ =='__main__':
    import timeit
    print(min(timeit.Timer('main0()', setup='from __main__ import main0').repeat(5,10)))

18.34290217700618


In [34]:
import numpy as np
import multiprocessing as mp

def main1():
    r, numThreads = np.random.normal(0, 0.01, size=(1000, 10000)), mp.cpu_count()
    parts = np.linspace(0, r.shape[0], min(numThreads, r.shape[0])+1)
    parts, jobs = np.ceil(parts).astype(int), []
    for i in range(1, len(parts)):
        jobs.append(r[:parts[i-1]:parts[i]]) # parallel jobs
        pool, out = mp.Pool(processes=numThreads), []
        outputs = pool.imap_unordered(barrierTouch, jobs)
        for out_ in outputs:
            out.append(out_)
        pool.close()
        pool.join()
    return out

if '__name__'=='__main__':
    import timeit
    print(min(timeit.Timer('main1()', setup='from __main__ import main1').repeat(5,10)))

In [35]:
def linParts(numAtoms, numTreads):
    parts = np.linspace(0, numAtoms, min(numTreads, numAtoms)+1)
    parts = np.ceil(parts).astype(int)
    return parts
linParts(1000, 6)

array([   0,  167,  334,  500,  667,  834, 1000])